In [6]:
import os
import glob
import re
import numpy as np
import numpy.fft as fft
# import mne
from numpy import save, load
import pandas as pd
import sklearn
# from sklearn.svm import SVC

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
from keras.optimizers import Adam
from keras.models import load_model
from keras.utils import np_utils


In [7]:
gpus=tf.config.experimental.list_physical_devices('GPU')
print("num gpus available", gpus)
tf.config.experimental.set_memory_growth(gpus[0], True)
print(tf.test.is_gpu_available())

num gpus available [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
True


### read dataset X and Y

In [9]:
subject_id=1
base_path = "features/"
# edf_file_names = sorted(glob.glob(os.path.join(base_path, "data_chb01/*.npy".format(subject_id))))
# files=len(edf_file_names)
X=load('features/data_chb01/features_{}_00.npy'.format(subject_id))
y=load('features/data_chb01/targets_{}_00.npy'.format(subject_id))
for subject_id in range(1,11):
    edf_file_names = sorted(glob.glob(os.path.join(base_path, "data_chb{:02d}/*.npy".format(subject_id))))
    files=len(edf_file_names)
    print(files)
    
    start=0
    if subject_id==1:
        start=1
    for fileno in range(start, files//2):
        X=np.concatenate((X, load('features/data_chb{:02d}/features_{}_{:02d}.npy'.format(subject_id, subject_id, fileno))))
        y=np.concatenate((y, load('features/data_chb{:02d}/targets_{}_{:02d}.npy'.format(subject_id,subject_id, fileno))))
    print(X.shape)


84
(14584, 208)
72
(27194, 208)
76
(40766, 208)
24
(55977, 208)
78
(70200, 208)
36
(94130, 208)
38
(118212, 208)
40
(125347, 208)
38
(149718, 208)
50
(167627, 208)


In [10]:
X_shape, y_shape = X.shape, y.shape
print(X_shape, y_shape)

(167627, 208) (167627,)


In [11]:
df=pd.DataFrame(data=X)
df['target']=y
print(df.shape)
df=df.dropna()
print(df.shape)

(167627, 209)
(167626, 209)


### train test split

In [12]:
df_interictal=df[df['target']==0]
# print(df_interictal.shape)
# df_interictal=df_interictal.sample(frac=0.7)
df_preictal=df[df['target']==2]
df_preictal = df_preictal.replace({'target': 2}, 1)
interictal_shape, preictal_shape = df_interictal.shape, df_preictal.shape
print(df_interictal.shape, df_preictal.shape)

(144326, 209) (8075, 209)


In [13]:
X_interictal=np.array(df_interictal[df.columns[:-1]]).astype('float32')
y_interictal=np.array(df_interictal['target']).astype('float32')
X_preictal  =np.array(df_preictal[df.columns[:-1]]).astype('float32')
y_preictal  =np.array(df_preictal['target']).astype('float32')

In [14]:
print(X_interictal.shape)
print(y_interictal.shape)
print(X_preictal.shape)
print(y_preictal.shape)

(144326, 208)
(144326,)
(8075, 208)
(8075,)


### train test split

In [15]:
from sklearn.model_selection import train_test_split


In [16]:
X_interictal_train, X_interictal_test, y_interictal_train, y_interictal_test =train_test_split(X_interictal,y_interictal,test_size=0.1, random_state=42)
X_preictal_train, X_preictal_test, y_preictal_train, y_preictal_test=train_test_split(X_preictal, y_preictal,test_size=0.1, random_state=42)

In [17]:
X_train = np.concatenate((X_interictal_train, X_preictal_train))
X_test = np.concatenate((X_interictal_test, X_preictal_test))
y_train = np.concatenate((y_interictal_train, y_preictal_train))
y_test = np.concatenate((y_interictal_test, y_preictal_test))

In [18]:
X_train_shape =X_train.shape
X_test_shape = X_test.shape
y_train_shape =y_train.shape 
y_test_shape = y_test.shape
print(X_train_shape)
print(X_test_shape)
print(y_train_shape)
print(y_test_shape)

(137160, 208)
(15241, 208)
(137160,)
(15241,)


### Normalization

In [19]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X)
X_train=scaler.transform(X_train)
X_test=scaler.transform(X_test)

### encoding

In [20]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

encoder = LabelEncoder()
# encoded_Y = encoder.fit_transform(y)
enc_y_train = np_utils.to_categorical(y_train)
enc_y_test = np_utils.to_categorical(y_test)
print(enc_y_test.shape)

(15241, 2)


### training the model

In [21]:
from datetime import datetime

In [22]:
model = keras.Sequential(
    [
        Dense(256, name="layer1", input_shape=(208,), activation = 'linear' ),
        Dense(256, name="hidden1", activation = 'linear'),
        Dense(128, name="hidden2", activation = 'linear'),
        Dense(128, name="hidden3", activation = 'linear'),
        Dense(64, name="hidden4", activation = 'linear'),
        Dense(32, name="hidden5", activation = 'linear'),
        Dense(2, name="output", activation = 'sigmoid'),
    ]
)
model.summary()

In [1]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy', tf.keras.metrics.FalseNegatives(thresholds=None, name=None, dtype=None)
])

NameError: name 'model' is not defined

In [31]:
before = datetime.now()
before_time =before.strftime("%H:%M:%S")
print(before_time)
class_weight = {0: 0.1, 1: 0.9}
model.fit(X_train, enc_y_train, epochs=1, batch_size= 32, validation_data=(X_test, enc_y_test), class_weight=class_weight)

after = datetime.now()
after_time =after.strftime("%H:%M:%S")
print(after_time)

print ("saving new model")
model.save("DNN7.pkl")

01:41:36
7694/7694 [==============================] - 16s 2ms/step - loss: 0.0962 - accuracy: 0.7580 - false_negatives_1: 28844.3531 - val_loss: 0.4838 - val_accuracy: 0.7570 - val_false_negatives_1: 6400.0000
01:41:52
saving new model
INFO:tensorflow:Assets written to: DNN7.pkl\assets


In [ ]:
model.evaluate(X_test, y_test)

In [38]:
# loaded_clf = joblib.load("my_models/SVM_blanced_chb04.pkl")
# y_pred=loaded_clf.predict(X_test)
y_pred1=model.predict(X_test)
y+pred1

In [39]:
y_pred=[]
for pr in y_pred1:
    y_pred=np.argmax(pr)
y_pred=np.array(y_pred)



In [73]:
# loaded_clf.get_params(deep=True)

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': 'balanced',
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [41]:
tn, fp, fn, tp = sklearn.metrics.confusion_matrix(y_test, [y_pred]).ravel()
print("TN:{}, FP:{}, FN:{}, TP:{}".format(tn, fp, fn, tp))

ValueError: Found input variables with inconsistent numbers of samples: [27357, 1]

In [26]:
specificity=(tn)/(tn+fp)
sensitivity=(tp)/(tp+fn)
print('specificity= {} , sensitivity= {}'.format(specificity, sensitivity))

specificity= 0.8802296964725185 , sensitivity= 0.9897959183673469


In [28]:
print('X size ={}, y size = {}'.format(X_shape, y_shape))
print('interictal size ={}, preictal size = {}'.format(interictal_shape, preictal_shape))
print('train size ={}, test size = {}'.format(X_train_shape, X_test_shape))

print("Training Accuracy: %d"%(trainAcc*100)+"%")
print("Testing Accuracy: %d"%(testAcc *100)+"%")
print("TN:{}, FP:{}, FN:{}, TP:{}".format(tn, fp, fn, tp))
print('specificity= {} , sensitivity= {}'.format(specificity, sensitivity))

X size =(14584, 208), y size = (14584,)
interictal size =(12188, 209), preictal size = (975, 209)
train size =(11846, 208), test size = (1317, 208)
Training Accuracy: 87%
Testing Accuracy: 88%
TN:1073, FP:146, FN:1, TP:97
specificity= 0.8802296964725185 , sensitivity= 0.9897959183673469


### grid search

In [24]:
# from numpy import mean
# from sklearn.model_selection import GridSearchCV
# from sklearn.model_selection import RepeatedStratifiedKFold

In [29]:
# balance = [{0:10,2:1}, {0:1,2:1}, {0:1,2:10}, {0:1,2:50}, {0:1,2:100}]
# costs=[1.0,10.0,100.0]
# param_grid = dict(C=costs, class_weight=balance)

In [36]:
# cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=1)
# grid = GridSearchCV(estimator=clf, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='f1_weighted')


In [37]:
# grid_result = grid.fit(X_train, y_train)

In [38]:
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# means = grid_result.cv_results_['mean_test_score']
# stds = grid_result.cv_results_['std_test_score']
# params = grid_result.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.897876 using {'C': 100.0, 'class_weight': {0: 1, 2: 10}}
0.890373 (0.000291) with: {'C': 1.0, 'class_weight': {0: 10, 2: 1}}
0.890373 (0.000291) with: {'C': 1.0, 'class_weight': {0: 1, 2: 1}}
0.749362 (0.010585) with: {'C': 1.0, 'class_weight': {0: 1, 2: 10}}
0.651175 (0.005219) with: {'C': 1.0, 'class_weight': {0: 1, 2: 50}}
0.631188 (0.009186) with: {'C': 1.0, 'class_weight': {0: 1, 2: 100}}
0.890373 (0.000291) with: {'C': 10.0, 'class_weight': {0: 10, 2: 1}}
0.890373 (0.000291) with: {'C': 10.0, 'class_weight': {0: 1, 2: 1}}
0.855444 (0.006939) with: {'C': 10.0, 'class_weight': {0: 1, 2: 10}}
0.784134 (0.010316) with: {'C': 10.0, 'class_weight': {0: 1, 2: 50}}
0.750769 (0.007781) with: {'C': 10.0, 'class_weight': {0: 1, 2: 100}}
0.890373 (0.000291) with: {'C': 100.0, 'class_weight': {0: 10, 2: 1}}
0.890787 (0.000550) with: {'C': 100.0, 'class_weight': {0: 1, 2: 1}}
0.897876 (0.006056) with: {'C': 100.0, 'class_weight': {0: 1, 2: 10}}
0.849596 (0.008939) with: {'C': 100.0, 'c